# for deno developers
deno install npm:groq-sdk
deno install npm:mongodb
# for node developers
you should adjust the code
npm install npm:groq-sdk
npm install npm:mongodb



In [ ]:
import "jsr:@std/dotenv/load";
// Import the MongoClient and ObjectId from the npm mongodb package
import { MongoClient, ObjectId } from "npm:mongodb";

import Groq from "npm:groq-sdk";
const apiKey = Deno.env.get("GROQ_API_KEY")||""; 

const groq = new Groq({ apiKey});
const  systemIsntruction=`
Vous êtes un agent de modération pour une plateforme de petites annonces. Votre but est de valider ou de rejeter une annonce basée sur son contenu."

Instructions :

Valider une annonce si elle est clairement liée à la vente ou à la location d'un article ou un service tels que la réparation des electricite des batiments

Rejeter une annonce si :

Le contenu est de nature politique.

Il ne s'agit pas d'une annonce de vente ou de location ou un service

Le contenu est offensant, illégal ou inapproprié.

Format de sortie : "Répondez avec un seul mot : 'PUBLIER' si l'annonce est valide, sinon 'REJETER'.

`

export async function getClassifationForAnnonce(annonceDesciption:string) {
  return await groq.chat.completions.create({
    messages: [
        {
      role: "system",
      content: systemIsntruction,
    },
      {
        role: "user",
        content:annonceDesciption
      },
    ],
    model: "openai/gpt-oss-120b", 
  });
}


// Connection string for local MongoDB
const uri = "mongodb://localhost:27017/";

// Database name
const dbName = "rim-ebay";

// Collection name
const collectionName = "Annonce";

// The specific ID to retrieve
const annonceId = "687f5682b9f671ec9d6ef516";

// Async function to connect and retrieve the specific document
async function getAnnonceById(
  uri:string, dbName:string, collectionName:string,
  annonceId:string) {
  // Create a new MongoClient
  const client = new MongoClient(uri);

  try {
    // Connect to the MongoDB server
    await client.connect();
    console.log("Connected successfully to MongoDB server");

    // Select the database
    const db = client.db(dbName);

    // Select the collection
    const collection = db.collection(collectionName);

    // Find the document by _id
    const document = await collection.findOne({ _id: new ObjectId(annonceId) });

    if (document) {
      console.log("Retrieved document:"); 
     console.log(document.description);
     const description=String(document.description);
     if(!description){
      console.log("No description found in the document.");
      return;
     }
     return description;
   
    } else {
      console.log("No document found with the given ID.");
      return
    }

  } catch (error) {
    console.error("Error connecting to MongoDB or fetching data:", error);
  } finally {
    // Close the connection
    await client.close();
    console.log("Connection closed");
  }
}

// Run the main function
const description = await getAnnonceById(annonceId,uri, dbName, collectionName);
if(description){
  const classification = await getClassifationForAnnonce(description);
    console.log("Classification result:", classification.choices[0]?.message?.content || "");
  const result=classification.choices[0]?.message?.content || "";
  if(result.includes("PUBLIER")){
    console.log("L'annonce est validée pour publication.");
  }else{
    console.log("L'annonce est rejetée.");
  }
}

Connected successfully to MongoDB server
Retrieved document:
feeeeeeeeeeer
Connection closed
Classification result: REJETER
L'annonce est rejetée.
